Created on Monday Jan 03 10:00:00 2022

@author: RAVENDRA KUMAR

python_version~ 3.8.5

# Process
* STEP1: Installing the required packages 
* STEP 2: Importing the required packages 
* STEP 3: Importing the data and processing it and Check the data and understanding the individal variables (Though in our case we have data which is already processed )
* STEP 4: Defining the Problem matrix: P and q in 1/2 X*PX - q X s.t. l<= Ax <= u
* STEP 5: Defining the contrained matrix A,l, and u
* STEP 6: Upload these matrices in the the OSQP library 
* STEP 7: Store the result

# Packages installation

In [17]:
# !pip install numpy 
# !pip install osqp
# !pip install pandas

# Main Code

In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import osqp
from scipy import sparse 

## Loading Raw Data

In [19]:
df = pd.read_excel("C:/Users/JMD/Dropbox/private_job/S&P_Global/test_input_file.xlsx", sheet_name=3)
df.columns = df.columns.str.replace(' ','_')
df.columns = map(str.lower, df.columns)
df = df.rename(columns={"stock_key": "stockkey" })
df

,stockkey,country,industry_group,6m_mdvt_(usd_mn),factor_score_1_eligibility_flag,factor_score_2,underlying_index_weight,liquidity_flag,eligible
0,42819,CN,5020,1685.065700,True,25.247578,0.062124,True,True
1,359307,CN,2550,4154.540850,True,58.086454,0.057374,True,True
2,11763,TW,4530,789.478600,True,319.255221,0.053561,True,True
3,8026583,CN,2550,919.254300,True,13.963829,0.022052,True,True
4,11557,ZA,2550,243.568740,True,120.570174,0.014327,True,True
...,...,...,...,...,...,...,...,...,...
1579,8111807,CN,2550,2.436551,False,64.087000,0.000010,True,False
1580,8111233,CN,3020,17.170643,False,407.132803,0.000010,True,False
1581,8111827,CN,4510,21.202158,True,19.695921,0.000010,True,True
1582,8107206,CN,1010,5.263357,False,556.969919,0.000008,True,False


In [20]:
df.isnull().sum()

stockkey                           0
country                            0
industry_group                     0
6m_mdvt_(usd_mn)                   0
factor_score_1_eligibility_flag    0
factor_score_2                     0
underlying_index_weight            0
liquidity_flag                     0
eligible                           0
dtype: int64

In [21]:
df.describe()

,stockkey,industry_group,6m_mdvt_(usd_mn),factor_score_2,underlying_index_weight
count,1.584000e+03,1584.000000,1584.000000,1584.000000,1584.000000
mean,8.990587e+05,3374.236111,62.353106,666.433542,0.000631
std,2.296601e+06,1390.269696,210.675191,2311.984427,0.002765
min,7.250000e+02,1010.000000,0.008791,1.845292,0.000006
25%,2.668975e+04,2030.000000,4.650715,32.118941,0.000079
50%,5.084550e+04,3520.000000,17.959315,93.373387,0.000203
75%,2.988102e+05,4510.000000,57.381543,364.665159,0.000492
max,8.177914e+06,6010.000000,5378.462700,43109.763281,0.062124


## Subsetting Eligible Universe

In [22]:
dataframe= df.copy() # Storing df as raw data 
dataframe.dtypes
dataframe["eligible"].unique()
dataframe["eligible"]= dataframe["eligible"].astype(str) # True and False are boolen data type hence needs to change their format
dataframe= dataframe[dataframe.eligible =="True"] # Keeping only eligible universe
dataframe=dataframe.reset_index() 
dataframe=dataframe.drop(dataframe.columns[0], axis=1)
dataframe=dataframe.iloc[:,0:7]
dataframe

,stockkey,country,industry_group,6m_mdvt_(usd_mn),factor_score_1_eligibility_flag,factor_score_2,underlying_index_weight
0,42819,CN,5020,1685.065700,True,25.247578,0.062124
1,359307,CN,2550,4154.540850,True,58.086454,0.057374
2,11763,TW,4530,789.478600,True,319.255221,0.053561
3,8026583,CN,2550,919.254300,True,13.963829,0.022052
4,11557,ZA,2550,243.568740,True,120.570174,0.014327
...,...,...,...,...,...,...,...
1003,682027,CN,4010,22.578463,True,6.520881,0.000018
1004,17398,CN,1010,10.133718,True,215.458070,0.000017
1005,762165,CN,3510,17.279327,True,65.137978,0.000013
1006,8111846,CN,3520,4.082206,True,48.480707,0.000012


## Defining the Problem matrix
writing the problem in following form 

$\min \dfrac{1}{2}X^TPX  + qX$

s.t. $ l \le AX \le u$

### Defining Objective (P and q)

In [23]:
a = np.zeros((1008, 1008), int) # Create matrix with only 0
np.fill_diagonal(a, 2) # fill diagonal with 2

P= pd.DataFrame(a) #Creating Dataframe using P matrix

dataframe_pwm= dataframe[['underlying_index_weight']]
dataframe_pwm.reset_index(inplace = True)
dataframe_pwm= dataframe_pwm[["underlying_index_weight"]]
dataframe_pwm= pd.merge(dataframe_pwm, P, left_index=True, right_index=True) # Merging using unique index

P_cols= P.columns
for i in P_cols:
    dataframe_pwm[i]= dataframe_pwm[i]/dataframe_pwm['underlying_index_weight']
    
P = dataframe_pwm.iloc[:,1:] # dropping the underlying_index_weight column


# renaming the columns as our desired variables

columns_name= []
for i in range(1,1009,1):
    x= "x_"+ str(i)
    #print(x)
    columns_name.append(x)    
P.columns=columns_name 


# Defining the linear part involved in the problem matrix 

q_linear_element= np.array([2 for x in range(1008)])

print('P:')
display(P)
print('\n\nq:')
display(q_linear_element)

P:


,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,x_10,...,x_999,x_1000,x_1001,x_1002,x_1003,x_1004,x_1005,x_1006,x_1007,x_1008
0,32.193518,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.00000
1,0.000000,34.859074,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.00000
2,0.000000,0.000000,37.340564,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.00000
3,0.000000,0.000000,0.000000,90.695909,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.00000
4,0.000000,0.000000,0.000000,0.000000,139.601258,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,113517.028619,0.00000,0.000000,0.000000,0.00000
1004,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,116025.63569,0.000000,0.000000,0.00000
1005,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,157845.908752,0.000000,0.00000
1006,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.000000,165883.946912,0.00000




q:


array([2, 2, 2, ..., 2, 2, 2])

### Defining contraints (A, l, u)

In [24]:
# Creating contraints for industry group 

indgrp= dataframe["industry_group"].unique()
indgrp_n=[]
for i in indgrp:
    i= str(i)
    indgrp_n.append(i)

for i in indgrp_n:
    dataframe[i]=i
    #print(len(dataframe.columns))
    
dataframe["industry_group"]= dataframe["industry_group"].astype(str)


# Assinging column value 1 if rows matches with the column otherwise filling with zero.

for i in indgrp_n:
    dataframe[i]= np.where(i==dataframe["industry_group"], 1,0)
    j="indgrp_"+i
    dataframe=dataframe.rename(columns={i:j})
   # print(i)
    

# Creating contraints for country

country= dataframe["country"].unique()
for i in country:
    dataframe[i]=i
    #print(len(dataframe.columns))
    
    
# Assinging column value 1 if rows matches with the column otherwise filling with zero.

for i in country:
    dataframe[i]= np.where(i==dataframe["country"], 1,0)
    j="country_"+i
    dataframe=dataframe.rename(columns={i:j})

    
# Creating contraints for stockkeys

dataframe["stockkey"]= dataframe["stockkey"].astype(str)   
stockkey= dataframe["stockkey"].unique()

for i in stockkey:
    dataframe[i]= i
    #print(len(dataframe.columns))

# Assinging column value 1 if rows matches with the column otherwise filling with zero

for i in stockkey:
    dataframe[i]= np.where(i==dataframe["stockkey"], 1,0)
    j="stockkey_"+i
    dataframe=dataframe.rename(columns={i:j})
    
    
dfa=dataframe # Storing dataframe for cross verification and understanding in case of any doubt in calculation 


# Lower Bound constraints formation 

list_lower_bound_max=[]         # Generating Lower bound list to store upper bound for each category
for i in dfa.columns[7:]:
    j=i.split("_")[0]
    if j== "indgrp" or j=="country":             
        dfa[i]= dfa[i].astype(int)  
        lbmp=sum(dfa["underlying_index_weight"]*dfa[i])
        temp= lbmp
        lbmp=max(lbmp-0.05, lbmp*.75) #defined in the methodology sheet
        tuplet= [i, lbmp]
        list_lower_bound_max.append(tuplet)
        #print(i,lbmp)
    else:
        dfa[i]= dfa[i].astype(int)  
        lbmp=sum(dfa["underlying_index_weight"]*dfa[i])
        lbmp=max(0,lbmp-0.02)
        tuplet= [i, lbmp]
        list_lower_bound_max.append(tuplet)
        #print(i,lbmp)

        
# Upper Bound constraints formation 

list_upper_bound_min=[]        # Generating Upper bound list to store upper bound for each category
for i in dfa.columns[7:]:
    j=i.split("_")[0]
    if j== "indgrp" or j=="country":          
        dfa[i]= dfa[i].astype(int)  
        lbmp=sum(dfa["underlying_index_weight"]*dfa[i])
        temp= lbmp
        lbmp=min(lbmp+0.05, lbmp*1.25) #defined in methodology sheet
        tuplet= [i, lbmp]
        list_upper_bound_min.append(tuplet)
        #print(i,lbmp)
    else:
        dfa[i]= dfa[i].astype(int)  
        lbmp=sum(dfa["underlying_index_weight"]*dfa[i])
        lbmp=min(0.08,lbmp*10,lbmp+.02)
        tuplet= [i, lbmp]
        list_upper_bound_min.append(tuplet)
        #print(i,lbmp)   

lower_bound_max=pd.DataFrame(list_lower_bound_max)  #Creating dataframe for lower bound for each category
lower_bound_max=lower_bound_max.transpose() 
lower_bound_max.columns=lower_bound_max.iloc[0]
lower_bound_max=lower_bound_max[1:]
upper_bound_min=pd.DataFrame(list_upper_bound_min)
upper_bound_min=upper_bound_min.transpose()
upper_bound_min.columns=upper_bound_min.iloc[0]
upper_bound_min=upper_bound_min[1:]

# Defining first constraint 

dfa["WAFS_2"] = dfa['factor_score_2']*dfa['underlying_index_weight']
lwr_upr_wafs_2=[0, sum(dfa["WAFS_2"])*.50]
WAFS_2_constraint = dfa['WAFS_2'].tolist()
WAFS_2_constraint= WAFS_2_constraint+ lwr_upr_wafs_2
WASF_2_constraint= pd.DataFrame(WAFS_2_constraint, columns=["wafs_2_target"])
dfa=dfa.drop(dfa.columns[-1], axis=1)


#dfa.underlying_index_weight.dtypes

dfa= pd.concat([dfa,lower_bound_max])  # Generating Lower bound list to store upper bound for each category
dfa= pd.concat([dfa,upper_bound_min])  # Generating Upper bound list to store upper bound for each category

dfa = dfa.reset_index()
dfa= dfa.drop(dfa.columns[0], axis=1)
dfa=pd.merge(dfa, WASF_2_constraint, left_index=True, right_index=True)

A= dfa.iloc[:,7:] # Dropping out first 7 columns as we do not need them anymore. 

A["hundred_percent_weight"]= 1  # Creating a column which says that sum of resulting weights must be 1. 
A= A.transpose()

l = A.iloc[:,-2] 
u = A.iloc[:,-1]
A = A.iloc[:,:-2]


print('A:')
display(A)
print('\n\n bounds: [lower, upper]')
display(np.c_[l, u])

#Checking for wrong calculation: lower bound must be lower than upper bound for each constraint
(l>u).pipe(lambda s: s[s]) # This should be empty

A:


,0,1,2,3,4,5,6,7,8,9,...,998,999,1000,1001,1002,1003,1004,1005,1006,1007
indgrp_5020,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
indgrp_2550,0,1,0,1,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
indgrp_4530,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
indgrp_1010,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
indgrp_4010,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
stockkey_762165,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
stockkey_8111846,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
stockkey_8111827,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
wafs_2_target,1.56849,3.33264,17.0996,0.307926,1.72735,6.67751,0.06961,0.648741,0.323132,0.0632334,...,0.0109613,0.00162108,0.00593901,0.000395107,0.00342204,0.000114888,0.00371397,0.000825336,0.000584514,0.000192635




 bounds: [lower, upper]


array([[0.06654572574820619, 0.110909542913677],
       [0.09291030788913641, 0.1548505131485607],
       [0.058289942580220064, 0.09714990430036678],
       ...,
       [0.0, 9.78047349663282e-05],
       [0.0, 157.22838842837703],
       [1, 1]], dtype=object)

Series([], dtype: bool)

## Solving Problem

In [25]:
# Convert problem/ Constraint dataframe into array form . 

P = P.values
A = A.values

c,b=np.linalg.eig(P) #checking positive semidefinite of problem matrix. If c>= 0 then P is positive-semidefinite matrix

q=np.array(q_linear_element)
l=np.array(l)
u=np.array(u)

P = sparse.csc_matrix(P)
A = sparse.csc_matrix(A.astype(float))


# Create an OSQP object

prob = osqp.OSQP()


# Setup workspace and change alpha parameter

prob.setup(P, q, A, l, u, alpha=1.6)

-----------------------------------------------------------------
           OSQP v0.6.2  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2021
-----------------------------------------------------------------
problem:  variables n = 1008, constraints m = 1059
          nnz(P) + nnz(A) = 6048
settings: linear system solver = qdldl,
          eps_abs = 1.0e-03, eps_rel = 1.0e-03,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 4000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: off, time_limit: off



In [26]:
res = prob.solve()

iter   objective    pri res    dua res    rho        time
   1  -1.9939e-03   1.00e+00   3.21e+03   1.00e-01   4.49e-03s
  50   3.1307e+00   6.83e-10   2.89e-08   1.00e-01   1.62e-02s

status:               solved
number of iterations: 50
optimal objective:    3.1307
run time:             1.69e-02s
optimal rho estimate: 3.47e-02



## Storing Solution

In [27]:
final_weights= res.x
final_weights=pd.DataFrame(final_weights, columns= ["final_weight"])
dataframe=dataframe.iloc[:,0:7] 
final_result= pd.merge(dataframe,final_weights, left_index=True, right_index=True)
final_result

,stockkey,country,industry_group,6m_mdvt_(usd_mn),factor_score_1_eligibility_flag,factor_score_2,underlying_index_weight,final_weight
0,42819,CN,5020,1685.065700,True,25.247578,0.062124,0.070033
1,359307,CN,2550,4154.540850,True,58.086454,0.057374,0.064677
2,11763,TW,4530,789.478600,True,319.255221,0.053561,0.060709
3,8026583,CN,2550,919.254300,True,13.963829,0.022052,0.024859
4,11557,ZA,2550,243.568740,True,120.570174,0.014327,0.016239
...,...,...,...,...,...,...,...,...
1003,682027,CN,4010,22.578463,True,6.520881,0.000018,0.000020
1004,17398,CN,1010,10.133718,True,215.458070,0.000017,0.000019
1005,762165,CN,3510,17.279327,True,65.137978,0.000013,0.000014
1006,8111846,CN,3520,4.082206,True,48.480707,0.000012,0.000014


In [28]:
final_result['underlying_index_weight'].sum()

0.884388688997294

In [29]:
final_result['final_weight'].sum()

1.0000000006827763

In [31]:
final_result.to_csv("C:/Users/JMD/Dropbox/private_job/S&P_Global/final_result.csv", index= False)